In [1]:
from collections import deque

# ---------- Rules & helpers ----------
# State is (missionaries_left, cannibals_left, boat_position)
# boat_position: 1 -> boat on LEFT (starting side)
#                0 -> boat on RIGHT (other side)
TOTAL = 3  # total missionaries/cannibals

def is_valid_state(state):
    missionaries_left, cannibals_left, _ = state
    missionaries_right = TOTAL - missionaries_left
    cannibals_right = TOTAL - cannibals_left

    # If there are missionaries present and cannibals outnumber them -> invalid
    if (missionaries_left > 0 and cannibals_left > missionaries_left) or \
       (missionaries_right > 0 and cannibals_right > missionaries_right):
        return False
    return True

def get_successors(state):
    successors = []
    m_left, c_left, boat_pos = state
    # If boat on left (1) it goes left->right so left side decreases: direction = -1
    # If boat on right (0) it goes right->left so left side increases: direction = +1
    direction = -1 if boat_pos == 1 else 1

    # allowed moves: (missionaries, cannibals)
    possible_moves = [(1,0), (0,1), (1,1), (2,0), (0,2)]

    for dm, dc in possible_moves:
        new_m_left = m_left + direction * dm
        new_c_left = c_left + direction * dc
        new_boat = 1 - boat_pos
        new_state = (new_m_left, new_c_left, new_boat)

        if 0 <= new_m_left <= TOTAL and 0 <= new_c_left <= TOTAL and is_valid_state(new_state):
            successors.append(new_state)

    return successors

# ---------- BFS solver ----------
def bfs_solve(start, goal):
    """Return path (list of states) from start to goal using BFS, or None if no path."""
    queue = deque([start])
    parent = {start: None}           # child -> parent
    move_from_parent = {}           # child -> (moved_missionaries, moved_cannibals)

    while queue:
        cur = queue.popleft()
        if cur == goal:
            # reconstruct path
            path = []
            node = cur
            while node is not None:
                path.append(node)
                node = parent[node]
            path.reverse()
            return path, move_from_parent

        for succ in get_successors(cur):
            if succ not in parent:   # not visited
                parent[succ] = cur
                moved_m = abs(cur[0] - succ[0])
                moved_c = abs(cur[1] - succ[1])
                move_from_parent[succ] = (moved_m, moved_c)
                queue.append(succ)

    return None, None

# ---------- Pretty printing ----------
def describe_state(s):
    left = f"{s[0]}M, {s[1]}C"
    right = f"{TOTAL - s[0]}M, {TOTAL - s[1]}C"
    boat_side = "Left" if s[2] == 1 else "Right"
    return f"Left: {left}  |  Right: {right}  |  Boat: {boat_side}"

def print_solution(path, move_map):
    print("\nSolution (step-by-step):")
    print("Start ->", describe_state(path[0]))
    for i in range(len(path) - 1):
        cur = path[i]
        nxt = path[i + 1]
        m, c = move_map[nxt]
        direction = "Left -> Right" if cur[2] == 1 else "Right -> Left"
        print(f"\nStep {i+1}: Move {m} missionary(ies) and {c} cannibal(s) {direction}")
        print("  Result ->", describe_state(nxt))
    print(f"\nTotal moves: {len(path)-1}")

# ---------- Interactive play ----------
def get_user_input(state):
    print("\nCurrent:", describe_state(state))
    print("Allowed moves: (missionaries cannibals) -> (1 0), (0 1), (1 1), (2 0), (0 2)")
    try:
        text = input("Your move (e.g. '1 1'): ").strip()
        m, c = map(int, text.split())
    except Exception:
        print("Invalid input format. Enter two integers separated by a space.")
        return None

    if (m, c) not in [(1,0),(0,1),(1,1),(2,0),(0,2)]:
        print("Move not allowed. Use one of the allowed moves.")
        return None

    # Check whether the move is possible given where the boat is and counts
    boat_direction = -1 if state[2] == 1 else 1
    new_state = (state[0] + boat_direction * m, state[1] + boat_direction * c, 1 - state[2])
    if 0 <= new_state[0] <= TOTAL and 0 <= new_state[1] <= TOTAL and is_valid_state(new_state):
        return (m, c)
    else:
        print("That move would lead to an invalid state (out of range or unsafe). Try another.")
        return None

def play_missionaries_and_cannibals():
    state = (TOTAL, TOTAL, 1)
    goal = (0, 0, 0)
    visited = set([state])

    while state != goal:
        move = None
        while move is None:
            move = get_user_input(state)

        m, c = move
        boat_direction = -1 if state[2] == 1 else 1
        new_state = (state[0] + boat_direction * m, state[1] + boat_direction * c, 1 - state[2])

        if 0 <= new_state[0] <= TOTAL and 0 <= new_state[1] <= TOTAL and is_valid_state(new_state):
            state = new_state
            visited.add(state)
            if state == goal:
                print("\n🎉 Congratulations! You solved it:")
                print(describe_state(state))
                return
        else:
            print("Invalid move (unsafe or out of bounds). Try again.")

    print("\nGame ended.")

# ---------- Main ----------
if __name__ == "__main__":
    start = (TOTAL, TOTAL, 1)
    goal = (0, 0, 0)

    print("Missionaries & Cannibals — Auto solver + Interactive")
    print("1) Auto-solve (BFS, shortest solution)")
    print("2) Play interactively")
    choice = input("Choose 1 or 2 (default 1): ").strip()

    if choice == "2":
        play_missionaries_and_cannibals()
    else:
        path, move_map = bfs_solve(start, goal)
        if path:
            print_solution(path, move_map)
        else:
            print("No solution found.")


Missionaries & Cannibals — Auto solver + Interactive
1) Auto-solve (BFS, shortest solution)
2) Play interactively
Choose 1 or 2 (default 1): 1

Solution (step-by-step):
Start -> Left: 3M, 3C  |  Right: 0M, 0C  |  Boat: Left

Step 1: Move 1 missionary(ies) and 1 cannibal(s) Left -> Right
  Result -> Left: 2M, 2C  |  Right: 1M, 1C  |  Boat: Right

Step 2: Move 1 missionary(ies) and 0 cannibal(s) Right -> Left
  Result -> Left: 3M, 2C  |  Right: 0M, 1C  |  Boat: Left

Step 3: Move 0 missionary(ies) and 2 cannibal(s) Left -> Right
  Result -> Left: 3M, 0C  |  Right: 0M, 3C  |  Boat: Right

Step 4: Move 0 missionary(ies) and 1 cannibal(s) Right -> Left
  Result -> Left: 3M, 1C  |  Right: 0M, 2C  |  Boat: Left

Step 5: Move 2 missionary(ies) and 0 cannibal(s) Left -> Right
  Result -> Left: 1M, 1C  |  Right: 2M, 2C  |  Boat: Right

Step 6: Move 1 missionary(ies) and 1 cannibal(s) Right -> Left
  Result -> Left: 2M, 2C  |  Right: 1M, 1C  |  Boat: Left

Step 7: Move 2 missionary(ies) and 0 can